In [1]:
import pandas as pd
import numpy as np

In [2]:
# fs is the whole original schedule
fs = pd.read_csv("/Users/mintking/Downloads/flight_schedule.csv", sep=',')

In [3]:
fs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2364 entries, 0 to 2363
Data columns (total 14 columns):
flight_id                 2364 non-null int64
date                      2364 non-null object
domistic/international    2364 non-null object
flight_num                2364 non-null int64
de_ap                     2364 non-null int64
la_ap                     2364 non-null int64
de_time                   2364 non-null object
la_time                   2364 non-null object
aircraft_id               2364 non-null int64
aircraft_type             2364 non-null int64
passenger_num             2364 non-null int64
connect_passenger_num     2364 non-null int64
seat_num                  2364 non-null int64
important_factor          2364 non-null float64
dtypes: float64(1), int64(9), object(4)
memory usage: 258.6+ KB


In [4]:
# make copy of fs
fs_copy = fs.copy()

In [5]:
#change the type of departure time and landing time and date
fs_copy.de_time = pd.to_datetime(fs_copy.de_time)
fs_copy.la_time = pd.to_datetime(fs_copy.la_time)
fs_copy.date = pd.to_datetime(fs_copy.date)

In [6]:
# extract domestic flights on 06/05/2017
fs_0506 = fs_copy[fs_copy.date == pd.Timestamp(year=2017,month=5, day=6)][fs_copy['domistic/international'] == 'domestic']

/Users/mintking/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


In [7]:
len(fs_0506.flight_num.unique())

337

In [8]:
# flight id is unrepeatable
len(fs_0506.flight_id.unique())

461

In [9]:
# 创建表格
_test_dataset = pd.DataFrame()

In [10]:
fs_0506.head(5)

,flight_id,date,domistic/international,flight_num,de_ap,la_ap,de_time,la_time,aircraft_id,aircraft_type,passenger_num,connect_passenger_num,seat_num,important_factor
4,10,2017-05-06,domestic,345,50,48,2017-05-06 08:05:00,2017-05-06 11:00:00,40,1,100,0,119,1.0
5,6,2017-05-06,domestic,344,68,49,2017-05-06 11:15:00,2017-05-06 13:25:00,6,2,90,0,161,1.0
15,16,2017-05-06,domestic,349,50,72,2017-05-06 07:30:00,2017-05-06 10:25:00,94,2,131,0,161,1.0
24,25,2017-05-06,domestic,356,72,49,2017-05-06 15:15:00,2017-05-06 17:55:00,29,3,103,0,163,1.0
26,27,2017-05-06,domestic,364,50,72,2017-05-06 16:25:00,2017-05-06 19:25:00,101,2,92,0,161,1.0


In [11]:
# duplicate fs_0506
fs_0506_copy = fs_0506.copy()

In [12]:
fs_0506_copy = fs_0506_copy.set_index('flight_id')

In [13]:
fs_0506_copy.head()

,date,domistic/international,flight_num,de_ap,la_ap,de_time,la_time,aircraft_id,aircraft_type,passenger_num,connect_passenger_num,seat_num,important_factor
flight_id,,,,,,,,,,,,,
10,2017-05-06,domestic,345,50,48,2017-05-06 08:05:00,2017-05-06 11:00:00,40,1,100,0,119,1.0
6,2017-05-06,domestic,344,68,49,2017-05-06 11:15:00,2017-05-06 13:25:00,6,2,90,0,161,1.0
16,2017-05-06,domestic,349,50,72,2017-05-06 07:30:00,2017-05-06 10:25:00,94,2,131,0,161,1.0
25,2017-05-06,domestic,356,72,49,2017-05-06 15:15:00,2017-05-06 17:55:00,29,3,103,0,163,1.0
27,2017-05-06,domestic,364,50,72,2017-05-06 16:25:00,2017-05-06 19:25:00,101,2,92,0,161,1.0


In [14]:
# the path between flight1 and flight2
def ff_connect(dataset, f_id):
    de_ap1, de_time1 = dataset.loc[f_id, ['de_ap', 'de_time']] #提取对应ID的departure airpor和departure time
    # de_time1 = fs_0506_copy.loc[f_id, 'de_time']
    
    value = dataset.loc[dataset['la_ap']==de_ap1].loc[dataset['de_time'] < de_time1]
    return value.index

print(ff_connect(fs_0506_copy,27))
#print(ff_connect(1241, 2070))
#print(ff_connect(1241,519))

Int64Index([  50,   57,  101,  225,  388,  533,  586,  727,  731,  753, 1138,
               5,  844,  941, 1018, 1050, 1089, 1094, 1125, 1279, 1630, 2363],
           dtype='int64', name='flight_id')


In [15]:
indexes1 = fs_0506['flight_id'].values

In [16]:
# flight to flight pheromone matrix
ffp1 = pd.DataFrame(data=np.zeros((461, 461)), index=indexes1, columns=indexes1)

In [17]:
for index in indexes1:
    ffp1.loc[ff_connect(fs_0506_copy,index), index] = 1

In [18]:
ffp1

,10,6,16,25,27,35,39,48,50,53,...,2325,2332,2335,2336,2346,2347,2348,2349,2359,2363
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
50,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# the path between 
ffp1.loc[16,25]

1.0

In [20]:
# the unique aircraft id on 0506
a_index = fs_0506_copy.aircraft_id.unique()

In [21]:
a_index

array([ 40,   6,  94,  29, 101,  12,   7, 134,  45, 121,   5,  77, 131,
        33, 141,  10,  15,  90,  91,  19,  41,  30,  65,  48,  44,  73,
        53,  68,  69, 117,  58,  43,  92,  76, 108,  74, 133, 132, 126,
       128,  50,  22,  21,  35,  25,  28,  20,  63,  27,  26,  84,  88,
       109,  81,   9, 106,  71,  98,   2,  47,  46, 130,  42, 142, 124,
        59,   8, 103,  36,  93,  64, 135,  54, 127, 140,  96,  18,  31,
       119,  56,  49, 114,  87,  78,  70,  52,  55,  67,  37,  38, 137,
        72,  79,  39,  83,  62,  14, 102,  61,  51, 129,  80, 100, 139,
        34,   4,  60,  32,  86,  82, 116, 110,  75, 123, 125, 136, 120,
       112, 113,  66, 115])

In [22]:
len(a_index)

121

In [23]:
# establish a table to save initial airport on 0506 and number of seats of each aircraft.
# initial airport is the departure airport of earliest flight.
a_status = pd.DataFrame(data=np.zeros((121, 2)), index=a_index, columns=['initial_ap', 'seats'])

In [24]:
a_status_copy = a_status.copy()

In [28]:
%pdb

Automatic pdb calling has been turned OFF


In [26]:
fs_0506_copy.loc[a_index]

/Users/mintking/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  if __name__ == '__main__':


,date,domistic/international,flight_num,de_ap,la_ap,de_time,la_time,aircraft_id,aircraft_type,passenger_num,connect_passenger_num,seat_num,important_factor
flight_id,,,,,,,,,,,,,
40,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
6,2017-05-06,domestic,344.0,68.0,49.0,2017-05-06 11:15:00,2017-05-06 13:25:00,6.0,2.0,90.0,0.0,161.0,1.0
94,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
29,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
101,2017-05-06,domestic,369.0,68.0,50.0,2017-05-06 11:55:00,2017-05-06 14:10:00,101.0,2.0,132.0,0.0,161.0,1.0
12,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
7,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
134,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
45,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for index in a_index:
    a_status.loc[index,'seats']= fs_0506_copy[fs_0506_copy.aircraft_id == index].seat_num.values[0]
    #print(fs_0506_copy[fs_0506_copy.aircraft_id == index].seat_num.values[0])
    #print(a_status.loc[index,'seats'])

# group by aircraft_id

In [93]:
earliest_de_time = fs_0506_copy.groupby(['aircraft_id'])['de_time'].min().reset_index() # reset index 为了把series 变成df

In [131]:
a_status = fs_0506_copy.merge(earliest_de_time, on=['de_time', 'aircraft_id'])[['aircraft_id', 'de_ap', 'seat_num']]

In [134]:
#fs_0506_copy.merge(a_status, how='outer', suffixes=('_1', '_2'))

,date,domistic/international,flight_num,de_ap,la_ap,de_time,la_time,aircraft_id,aircraft_type,passenger_num,connect_passenger_num,seat_num,important_factor
0,2017-05-06,domestic,345,50,48,2017-05-06 08:05:00,2017-05-06 11:00:00,40,1,100,0,119,1.0
1,2017-05-06,domestic,252,50,31,2017-05-06 18:05:00,2017-05-06 19:40:00,40,1,39,0,119,1.0
2,2017-05-06,domestic,344,68,49,2017-05-06 11:15:00,2017-05-06 13:25:00,6,2,90,0,161,1.0
3,2017-05-06,domestic,349,50,72,2017-05-06 07:30:00,2017-05-06 10:25:00,94,2,131,0,161,1.0
4,2017-05-06,domestic,356,72,49,2017-05-06 15:15:00,2017-05-06 17:55:00,29,3,103,0,163,1.0
5,2017-05-06,domestic,364,50,72,2017-05-06 16:25:00,2017-05-06 19:25:00,101,2,92,0,161,1.0
6,2017-05-06,domestic,368,50,68,2017-05-06 08:50:00,2017-05-06 11:05:00,101,2,123,0,161,1.0
7,2017-05-06,domestic,357,50,67,2017-05-06 10:50:00,2017-05-06 13:20:00,12,2,126,0,161,1.0
8,2017-05-06,domestic,365,72,50,2017-05-06 20:25:00,2017-05-06 23:10:00,101,2,122,0,161,1.0
9,2017-05-06,domestic,354,71,32,2017-05-06 12:30:00,2017-05-06 14:15:00,7,2,81,39,161,1.0


In [148]:
a_status=a_status.set_index('aircraft_id')

In [150]:
def af_connect(a_id, dataset1, dataset2):
    de_ap1, seat_num1 = dataset1.loc[a_id, ['de_ap', 'seat_num']]
    value = dataset2.loc[dataset2['de_ap']==de_ap1].loc[dataset2['passenger_num'] <= seat_num1]
    
    return value.index
print(af_connect(90,a_status,fs_0506_copy))

    

Int64Index([  59,   74,   77,  136,  147,  155,  162,  174,  199,  203,  231,
             252,  282,  335,  394,  445,  496,  501,  526,  531,  536,  559,
             597,  655,  742, 1242, 1012,   43,  437,  669, 1115,  166, 1087,
             765, 1558,  871, 1809,  677,  814,  891,  863, 1163,  848, 1044,
            1205, 2103, 2262,  476, 1867, 2214,  242,  700,  776, 1040,  484,
             934, 2301, 1148, 1853, 1035,  480,  178, 1228, 1275, 1296, 1650,
            2116, 2346],
           dtype='int64', name='flight_id')


In [151]:
# aircraft to flight pheromone matrix
afp1 = pd.DataFrame(data=np.zeros((121, 461)), index=a_index, columns=indexes1)

In [152]:
afp1

,10,6,16,25,27,35,39,48,50,53,...,2325,2332,2335,2336,2346,2347,2348,2349,2359,2363
40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [153]:
a_status

,de_ap,seat_num
aircraft_id,,
40,50,119
94,50,161
12,50,161
29,49,163
45,49,119
101,50,161
131,32,183
141,72,183
10,62,161


In [154]:
for a_id in a_index:
    afp1.loc[a_id,af_connect(a_id, a_status,fs_0506_copy)] = 1

In [155]:
afp1

,10,6,16,25,27,35,39,48,50,53,...,2325,2332,2335,2336,2346,2347,2348,2349,2359,2363
40,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
101,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
12,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
